# Projeto Final de Manipulação de Dados

**OBJETIVO**: O objetivo é trabalhar os conceitos de manipulação de dados para processar as bases de Escola e de IDEB extraídas do INEP

---

## Cabeçalho 

In [ ]:
import os
import requests
import zipfile

import numpy as np
import pandas as pd

from io import BytesIO
from pathlib import Path
from pyarrow.lib import ArrowInvalid

In [ ]:
%config Completer.use_jedi = False

# pode ser "completo" ou "amostra"
TIPO_DADOS = "completo"

# checa se o notebook está sendo executado no google colab
GOOGLE_COLAB = "google.colab" in str(get_ipython())

# monta a pasta com os conteúdos
if GOOGLE_COLAB:
    from google.colab import drive
    drive.mount("/content/drive")
    
# informa o caminho para a pasta de ciência de dados a partir do drive
# você pode deixar isso como vazio se você não tiver adicionado a pasta
CAMINHO_DRIVE = "Ciência de Dados"

In [ ]:
vPATH_NOTEBOOK = Path(os.path.dirname(os.path.realpath("__file__")))
vPATH_DADOS = vPATH_NOTEBOOK.parent.parent / f"dados/{TIPO_DADOS}"

---

## Carregamento

Como primeiro passo, nós precisamos carregar os dados que nós vamos trabalhar. Neste caso nós vamos carregar duas bases de dados:

1. Base do censo escolar para os anos de 2017 e 2019
2. Base de nota de IDEB por escola e ano dos anos iniciais, anos finais e ensino médio

### Censo Escolar 

Para o primeiro, nós vamos carregar os dados contidos em https://drive.google.com/drive/folders/11hj0Q7ubVnmbmlE2M45n7A3HYTgHJcRm?usp=sharing, mas nós vamos selecionar apenas as colunas de:
- NU_ANO_CENSO: Ano de coleta do censo
- CO_ENTIDADE: Código da Escola
- NO_ENTIDADE: Nome da Escola
- TP_SITUACAO_FUNCIONAMENTO: Situação de Funcionamento (1=Em Atividade, 2=Paralisada, 3=Extinsta (ano do Censo), 4=Extinta em Anos Anteriores)
- DT_ANO_LETIVO_INICIO: Ínicio do ano letivo
- DT_ANO_LETIVO_TERMINO: Término (Previsão) do ano letivo
- CO_MUNICIPIO: Código IBGE do município
- TP_DEPENDENCIA: Depêndencia Administrativa (1=Federal, 2=Estadual, 3=Municipal, 4=Privada)
- IN_LOCAL_FUNC_PREDIO_ESCOLAR: Local de funcionamento da escola - Prédio Escolar
- IN_AGUA_INEXISTENTE: Abastecimento de água - Não há abastecimento de água
- IN_ENERGIA_INEXISTENTE: Abastecimento de energia elétrica - Não há energia elétrica
- IN_ESGOTO_INEXISTENTE: Esgoto sanitário - Não há esgotamento sanitário
- IN_ALMOXARIFADO: Dependências físicas existentes e utilizadas na escola - Almoxarifado
- IN_AUDITORIO: Dependências físicas existentes e utilizadas na escola - Auditório
- IN_BANHEIRO: Dependências físicas existentes e utilizadas na escola - Banheiro 
- IN_BIBLIOTECA: Dependências físicas existentes e utilizadas na escola - Biblioteca
- IN_SALA_LEITURA: Dependências físicas existentes e utilizadas na escola - Sala de Leitura
- IN_COZINHA: Dependências físicas existentes e utilizadas na escola - Cozinha 
- IN_REFEITORIO: Dependências físicas existentes e utilizadas na escola - Refeitório
- IN_LABORATORIO_CIENCIAS: Dependências físicas existentes e utilizadas na escola - Laboratório de ciências
- IN_LABORATORIO_INFORMATICA: Dependências físicas existentes e utilizadas na escola - Laboratório de informática
- IN_QUADRA_ESPORTES: Dependências físicas existentes e utilizadas na escola - Quadra de esportes coberta ou descoberta
- IN_EQUIP_PARABOLICA: Equipamentos existentes na escola - Antena parabólica
- IN_COMPUTADOR: Equipamentos existentes na escola - Computador
- IN_EQUIP_COPIADORA: Equipamentos existentes na escola - Copiadora
- IN_EQUIP_IMPRESSORA: Equipamentos existentes na escola - Impressora
- IN_EQUIP_DVD: Equipamentos existentes na escola para o processo ensino aprendizagem - DVD/Blu-ray
- IN_EQUIP_SOM: Equipamentos existentes na escola para o processo ensino aprendizagem - Aparelho de som
- IN_EQUIP_TV: Equipamentos existentes na escola para o processo ensino aprendizagem - Aparelho de televisão
- IN_EQUIP_MULTIMIDIA: Equipamentos existentes na escola para o processo ensino aprendizagem - Projetor Multimídia (Datashow)
- IN_INTERNET: Acesso à Internet
- IN_ALIMENTACAO: Alimentação escolar para os alunos - PNAE/FNDE


DICA (1): Dê uma olhada no atributo z.filelist do objeto zipfile

DICA (2): Dá pra abrir um arquivo zip dentro de um arquivo zip?

De o nome de **esc_2017** e **esc_2019** para os dados carregados

In [ ]:
colunas = [
    "NU_ANO_CENSO",
    "CO_ENTIDADE",
    "NO_ENTIDADE",
    "TP_SITUACAO_FUNCIONAMENTO",
    "DT_ANO_LETIVO_INICIO",
    "DT_ANO_LETIVO_TERMINO",
    "CO_MUNICIPIO",
    "TP_DEPENDENCIA",
    "IN_LOCAL_FUNC_PREDIO_ESCOLAR",
    "IN_AGUA_INEXISTENTE",
    "IN_ENERGIA_INEXISTENTE",
    "IN_ESGOTO_INEXISTENTE",
    "IN_ALMOXARIFADO",
    "IN_AUDITORIO",
    "IN_BIBLIOTECA",
    "IN_SALA_LEITURA",
    "IN_COZINHA",
    "IN_REFEITORIO",
    "IN_LABORATORIO_CIENCIAS",
    "IN_LABORATORIO_INFORMATICA",
    "IN_QUADRA_ESPORTES",
    "IN_EQUIP_PARABOLICA",
    "IN_COMPUTADOR",
    "IN_EQUIP_COPIADORA",
    "IN_EQUIP_IMPRESSORA",
    "IN_EQUIP_DVD",
    "IN_EQUIP_SOM",
    "IN_EQUIP_TV",
    "IN_EQUIP_MULTIMIDIA",
    "IN_INTERNET",
    "IN_ALIMENTACAO",
]

Google Colab

In [ ]:
#@title Resposta
colunas = [
    "NU_ANO_CENSO",
    "CO_ENTIDADE",
    "NO_ENTIDADE",
    "TP_SITUACAO_FUNCIONAMENTO",
    "DT_ANO_LETIVO_INICIO",
    "DT_ANO_LETIVO_TERMINO",
    "CO_MUNICIPIO",
    "TP_DEPENDENCIA",
    "IN_LOCAL_FUNC_PREDIO_ESCOLAR",
    "IN_AGUA_INEXISTENTE",
    "IN_ENERGIA_INEXISTENTE",
    "IN_ESGOTO_INEXISTENTE",
    "IN_ALMOXARIFADO",
    "IN_AUDITORIO",
    "IN_BIBLIOTECA",
    "IN_SALA_LEITURA",
    "IN_COZINHA",
    "IN_REFEITORIO",
    "IN_LABORATORIO_CIENCIAS",
    "IN_LABORATORIO_INFORMATICA",
    "IN_QUADRA_ESPORTES",
    "IN_EQUIP_PARABOLICA",
    "IN_COMPUTADOR",
    "IN_EQUIP_COPIADORA",
    "IN_EQUIP_IMPRESSORA",
    "IN_EQUIP_DVD",
    "IN_EQUIP_SOM",
    "IN_EQUIP_TV",
    "IN_EQUIP_MULTIMIDIA",
    "IN_INTERNET",
    "IN_ALIMENTACAO",
]

with zipfile.ZipFile(f"drive/MyDrive/{CAMINHO_DRIVE}/dados/{TIPO_DADOS}/externo/censo_escolar/2017.zip") as z:
    with zipfile.ZipFile(z.open("Microdados_Censo_Escolar_2017/DADOS/ESCOLAS.zip")) as z2:
        esc_2017 = pd.read_csv(
            z2.open("ESCOLAS.CSV"), 
            sep="|", 
            encoding="latin-1",
            usecols=colunas
        )

with zipfile.ZipFile(f"drive/MyDrive/{CAMINHO_DRIVE}/dados/{TIPO_DADOS}/externo/censo_escolar/2019.zip") as z:
    esc_2019 = pd.read_csv(
        z.open("microdados_educacao_basica_2019/DADOS/ESCOLAS.CSV"), 
        sep="|", 
        encoding="latin-1",
        usecols=colunas
    )

Local

In [ ]:
#@title Resposta
colunas = [
    "NU_ANO_CENSO",
    "CO_ENTIDADE",
    "NO_ENTIDADE",
    "TP_SITUACAO_FUNCIONAMENTO",
    "DT_ANO_LETIVO_INICIO",
    "DT_ANO_LETIVO_TERMINO",
    "CO_MUNICIPIO",
    "TP_DEPENDENCIA",
    "IN_LOCAL_FUNC_PREDIO_ESCOLAR",
    "IN_AGUA_INEXISTENTE",
    "IN_ENERGIA_INEXISTENTE",
    "IN_ESGOTO_INEXISTENTE",
    "IN_ALMOXARIFADO",
    "IN_AUDITORIO",
    "IN_BIBLIOTECA",
    "IN_SALA_LEITURA",
    "IN_COZINHA",
    "IN_REFEITORIO",
    "IN_LABORATORIO_CIENCIAS",
    "IN_LABORATORIO_INFORMATICA",
    "IN_QUADRA_ESPORTES",
    "IN_EQUIP_PARABOLICA",
    "IN_COMPUTADOR",
    "IN_EQUIP_COPIADORA",
    "IN_EQUIP_IMPRESSORA",
    "IN_EQUIP_DVD",
    "IN_EQUIP_SOM",
    "IN_EQUIP_TV",
    "IN_EQUIP_MULTIMIDIA",
    "IN_INTERNET",
    "IN_ALIMENTACAO",
]

with zipfile.ZipFile(vPATH_DADOS / "externo/censo_escolar/2017.zip") as z:
    with zipfile.ZipFile(z.open("Microdados_Censo_Escolar_2017/DADOS/ESCOLAS.zip")) as z2:
        esc_2017 = pd.read_csv(
            z2.open("ESCOLAS.CSV"), 
            sep="|", 
            encoding="latin-1",
            usecols=colunas
        )

with zipfile.ZipFile(vPATH_DADOS / "externo/censo_escolar/2019.zip") as z:
    esc_2019 = pd.read_csv(
        z.open("microdados_educacao_basica_2019/DADOS/ESCOLAS.CSV"), 
        sep="|", 
        encoding="latin-1",
        usecols=colunas
    )

### IDEB 

Vamos agora carregar os dados de IDEB. Note que neste caso os dados estão em um arquivo ZIP, mas são do formato excel. Neste caso nós também vamos carregar os dados na íntegra, mas note uma coisa no seu conteúdo: A primeira linha de dados acontece quando? Será que há algum parâmetro que nos permita determinar em que linha começar a carregar um arquivo?

Dê os nomes de **ideb_ai**, **ideb_af** e **ideb_em** para os dataframes carregados

Google Colab

In [ ]:
#@title Resposta
with zipfile.ZipFile(
    f"drive/MyDrive/{CAMINHO_DRIVE}/dados/{TIPO_DADOS}/externo/ideb/divulgacao_anos_finais_escolas_2019.zip"
) as z:
    ideb_af = pd.read_excel(
        z.open("divulgacao_anos_finais_escolas_2019/divulgacao_anos_finais_escolas_2019.xlsx"),
        skiprows=9
    )

with zipfile.ZipFile(
    f"drive/MyDrive/{CAMINHO_DRIVE}/dados/{TIPO_DADOS}/externo/ideb/divulgacao_anos_iniciais_escolas_2019.zip"
) as z:
    ideb_ai = pd.read_excel(
        z.open("divulgacao_anos_iniciais_escolas_2019/divulgacao_anos_iniciais_escolas_2019.xlsx"),
        skiprows=9
    )
    
with zipfile.ZipFile(
    f"drive/MyDrive/{CAMINHO_DRIVE}/dados/{TIPO_DADOS}/externo/ideb/divulgacao_ensino_medio_escolas_2019.zip"
) as z:
    ideb_em = pd.read_excel(
        z.open("divulgacao_ensino_medio_escolas_2019/divulgacao_ensino_medio_escolas_2019.xlsx"),
        skiprows=9
    )

Local

In [ ]:
#@title Resposta
with zipfile.ZipFile(vPATH_DADOS / "externo/ideb/divulgacao_anos_finais_escolas_2019.zip") as z:
    ideb_af = pd.read_excel(
        z.open("divulgacao_anos_finais_escolas_2019/divulgacao_anos_finais_escolas_2019.xlsx"),
        skiprows=9
    )

with zipfile.ZipFile(vPATH_DADOS / "externo/ideb/divulgacao_anos_iniciais_escolas_2019.zip") as z:
    ideb_ai = pd.read_excel(
        z.open("divulgacao_anos_iniciais_escolas_2019/divulgacao_anos_iniciais_escolas_2019.xlsx"),
        skiprows=9
    )
    
with zipfile.ZipFile(vPATH_DADOS / "externo/ideb/divulgacao_ensino_medio_escolas_2019.zip") as z:
    ideb_em = pd.read_excel(
        z.open("divulgacao_ensino_medio_escolas_2019/divulgacao_ensino_medio_escolas_2019.xlsx"),
        skiprows=9
    )

---

## Exploração De Dados 

### Escola 

Antes de fazermos qualquer tratamento de dados, vamos explorar esses dataframes e entender seu conteúdo. Vamos começar com algo simples, vamos ver as primeiro 5, as últimas 5 e 5 linhas aleatórias do dataframe

In [ ]:
#@title Resposta
display(esc_2019.head())
display(esc_2019.tail())
display(esc_2019.sample(5))

Bom, daqui nós já podemos fazer as primeiras anotações das mudanças que teremos de realizar nos dados:

1. As colunas TP_SITUACAO_FUNCIONAMENTO e TP_DEPENDENCIA estão codificadas, e seria útil termos algo que consegue ser lido e facilmente filtrado sem precisar decorar o que cada código significa
2. DT_ANO_LETIVO_INICIO e DT_ANO_LETIVO_TERMINO: São datas, mas estão codificadas como texto, nós podemos converte-las para um tipo que seja mais aderente com seu significado
3. Há nulos nas colunas IN_, seria bom entendermos porque esses dados não estão ai

Bom, vamos olhar mais de perto para esses dados, lista quais são as colunas disponíveis, seus tipos de dados, e os conteúdos nulos

In [ ]:
#@title Resposta
esc_2019.info(memory_usage="deep")

Daqui nós conseguimos ver que provavelmente nós podemos economizar memória ao converter as colunas IN e provavelmente também com CO_ENTIDADE e CO_MUNICIPIO

Vamos nos aprofundar um pouco mais. 

As colunas IN, segundo o dicionário de dados, possuem 3 valores possíveis: 0, 1 e 9.

Os dois primeiros são as flags que indicativas do campo, enquanto que o valor 9 deveria indicar que os dados não foram preenchidos.

Vamos fazer uma checagem do número de valores para cada campo IN. Imprima a quantidade de linhas com os valores únicos de cada coluna que começa com IN_

In [ ]:
#@title Resposta
cols_in = [c for c in esc_2019 if c.startswith("IN_")]
for c in cols_in:
    print(c)
    for v in sorted(list(esc_2019[c].unique())):
        if not pd.isnull(v):
            print(f"{v}: {esc_2019.loc[lambda f: f[c] == v].shape[0]}")
        else:
            print(f"{v}: {esc_2019.loc[lambda f: f[c].isnull()].shape[0]}")
    print("-" * 100)

Duas coisas deveriam chamar atenção nessa impressão:
1. Não há valores 9
2. Os valores de nulos são sempre iguais para cada campo

Provavelmente os nulos são para escolas que estavam estavam extintas ou paralisadas no ano do censo, mas chequemos isso: Realize o mesmo print acima só que apenas para escolas extintas ou paralizadas

In [ ]:
#@title Resposta
df = esc_2019.loc[lambda f: f["TP_SITUACAO_FUNCIONAMENTO"] > 1]
cols_in = [c for c in esc_2019 if c.startswith("IN_")]
for c in cols_in:
    print(c)
    for v in sorted(list(df[c].unique())):
        if not pd.isnull(v):
            print(f"{v}: {df.loc[lambda f: f[c] == v].shape[0]}")
        else:
            print(f"{v}: {df.loc[lambda f: f[c].isnull()].shape[0]}")
    print("-" * 100)

Bingo! Provavelmente nós poderemos remover essas escolas da base e converter esses campos para uint8 para reduzir memória

---

### IDEB 

Vamos começar fazendo algo similar, para cada um dos dataframes imprima as 5 primeiras, 5 últimas e 5 linhas aleatórias

In [ ]:
#@title Resposta
for nome, df in [("IDEB AI", ideb_ai), ("IDEB AF", ideb_af), ("IDEB EM", ideb_em)]:
    print(nome)
    display(df.head())
    display(df.tail())
    display(df.sample(5))
    print("-" * 100)

Aqui nós podemos notar alguns problemas mais graves com o dataframe:

1. As linhas finais não deveriam fazer parte dos dados carregados
2. Há traços no meuio das colunas numéricas
3. Cada ano esta registrado como uma coluna distinta para as mesmas métricas

Vamos olhar para os tipos de dado de cada coluna

In [ ]:
#@title Resposta
ideb_ai.info(memory_usage="deep")

Como nós podemos ver há muitas colunas nestes dataframes e a grande maioria delas são objetos, indicando que o "-" que foi adicionado as colunas núméricas vai gerar uma série de problemas, por exemplo, vamos tentar fazer um resumo estatístico do dataframe:

In [ ]:
#@title Resposta
ideb_ai.describe()

Vê? Nós só temos acesso as colunas de código (que não são nada úteis pra isso), o que gerará todo tipo de problema nas análises. Vamos listar todas as colunas que nós temos disponíveis

In [ ]:
#@title Resposta
print("| ".join(list(ideb_ai.columns)))
print()
print("| ".join(list(ideb_af.columns)))
print()
print("| ".join(list(ideb_em.columns)))

Interessante, aparentemento nós podemos ver que há 7 métricas:
1. VL_APROVACAO: Taxa de alunos aprovados para um determinado ano
1. VL_INDICADOR_REND: <a href="https://academia.qedu.org.br/censo-escolar/taxa-de-rendimento/?repeat=w3tc">Indicador de rendimento</a> 
1. VL_NOTA_MATEMATICA: Nota média de Matemática na Prova Brasil
1. VL_NOTA_PORTUGUES: Nota média de Português na Prova Brasil
1. VL_NOTA_MEDIA: Nota média final na Prova Brasil
1. VL_OBSERVADO: IDEB calculado para o ano
1. VL_PROJECAO: Meta de IDEB para o ano

Nós podemos ver que VL_APROVACAO ainda é quebrado por ano ou considerando a junção dos anos que compõe o conjunto de séries para o índice.

Além disso cada indicador vem com os anos 2005, 2007, 2009, 2011, 2013, 2015, 2017 e 2019, exceto o Ensino Médio que só possuí os anos de 2017 e 2019.

Por fim, nós podemos notar que nós temos a meta de IDEB para o ano de 2021, mas não temos nota porque (até a data de extração da base) nós não temos coleta dos dados

---

## Processamento de Dados 

Bom agora que nós olhamos para os dataframes nós podemos começar a fazer os ajustes de dados conforme o que nós decobrimos

### Escolas 

Vamos começar pelo óbvio: As bases de 2017 e 2019 tem as mesmas estruturas de dados, portanto faria sentido junta-las numa única base para que todos os processamentos de dados que ainda temos que fazer sejam feitos sobre essa base consolidada.

Crie uma base "escolas" que empilhe as bases de 2017 e 2019 e resete os índices dessa nova base

In [ ]:
#@title Resposta
escolas = esc_2017.append(esc_2019).reset_index(drop=True)

Como nós vimos escolas que estão extintas ou paralisadas possuirão uma série de informações faltantes, portanto não faz sentido mante-las na base.

Remova dos dados as escolas que estão nas condições acima

In [ ]:
#@title Resposta
escolas = escolas.loc[lambda f: f["TP_SITUACAO_FUNCIONAMENTO"] == 1]

Agora que nós fizemos isso não faz sentido manter o campo TP_SITUACAO_FUNCIONAMENTO dado que ele só possuí um único valor para todas as linhas, portanto remova esse campo da base

In [ ]:
#@title Resposta
escolas.drop(columns=["TP_SITUACAO_FUNCIONAMENTO"], inplace=True)

Primeiro, vamos fazer a tarefa mais simples: Converta os campos de DT_ANO_LETIVO_INICIO e DT_ANO_LETIVO_TERMINO para datetimes

In [ ]:
#@title Resposta
escolas = escolas.assign(
    DT_ANO_LETIVO_INICIO=lambda f: pd.to_datetime(f["DT_ANO_LETIVO_INICIO"], format="%d/%m/%Y"),
    DT_ANO_LETIVO_TERMINO=lambda f: pd.to_datetime(f["DT_ANO_LETIVO_TERMINO"], format="%d/%m/%Y"),
)

Daqui nós podemos gerar um calculo interessante. Talvez possa existir alguma correleção entre o número de dias letivos de uma escola com a nota final do IDEB.

Gere uma nova coluna QT_DIAS_LETIVOS que seja um número inteiro com os dias entre as datas de ínicio e fim do período letivo

NOTA: O calculo correto deveria levar em conta feriados e férias, mas por simplificação vamos supor que os dias letivos sejam os dias totais entre as datas

In [ ]:
#@title Resposta
escolas = escolas.assign(
    QT_DIAS_LETIVOS=lambda f: (f["DT_ANO_LETIVO_TERMINO"] - f["DT_ANO_LETIVO_INICIO"]).dt.days
)

Agora vamos ajustar as colunas TP, nós temos:

- TP_DEPENDENCIA: Depêndencia Administrativa (1=Federal, 2=Estadual, 3=Municipal, 4=Privada)

Converta os valores de números dessas colunas para textos.

NOTA: Considere Extinta (ano do Censo) = Extinta em Anos Anteriores = Extinta

In [ ]:
#@title Resposta
escolas = escolas.assign(
    TP_DEPENDENCIA=lambda f: f["TP_DEPENDENCIA"].replace({
        1: "Federal",
        2: "Estadual",
        3: "Municipal",
        4: "Privada",
    })
)

Agora que nós temos as colunas como textos vamos economizar memória. Converta as mesmas colunas anteriores para category

In [ ]:
#@title Resposta
escolas = escolas.assign(
    TP_DEPENDENCIA=lambda f: f["TP_DEPENDENCIA"].astype(
        pd.CategoricalDtype(categories=["Federal", "Estadual", "Municipal", "Privada"])
    ),
)

Com o dataframe neste estado, todas as colunas IN deveriam conter apenas os valores 0 e 1 e, portanto, podem ser convertidas para uint8. Realize essa conversão nesses campos

In [ ]:
#@title Resposta
cols_in = [c for c in esc_2019 if c.startswith("IN_")]
for c in cols_in:
    escolas[c] = escolas[c].astype("uint8")

---

### IDEB 

Para poder ajustar os dados das bases, primeiro nós temos que remover as linhas de dados finais da base que contém algumas notas sobre os dados coletados.

In [ ]:
#@title Resposta
ideb_ai.dropna(subset=list(ideb_ai.loc[:, "CO_MUNICIPIO":].columns), how="all", inplace=True)
ideb_af.dropna(subset=list(ideb_af.loc[:, "CO_MUNICIPIO":].columns), how="all", inplace=True)
ideb_em.dropna(subset=list(ideb_em.loc[:, "CO_MUNICIPIO":].columns), how="all", inplace=True)

Agora nós podemos realizar alterações estruturais os dataframes. No caso nós temos 7 métricas:

1. VL_APROVACAO: Taxa de alunos aprovados para um determinado ano
1. VL_INDICADOR_REND: <a href="https://academia.qedu.org.br/censo-escolar/taxa-de-rendimento/?repeat=w3tc">Indicador de rendimento</a> 
1. VL_NOTA_MATEMATICA: Nota média de Matemática na Prova Brasil
1. VL_NOTA_PORTUGUES: Nota média de Português na Prova Brasil
1. VL_NOTA_MEDIA: Nota média final na Prova Brasil
1. VL_OBSERVADO: IDEB calculado para o ano
1. VL_PROJECAO: Meta de IDEB para o ano

Vamos fazer com que cada uma dessas métricas tenha uma única coluna (exceto VL_APROVACAO em que há a abertura por anos) e que os anos sejam colocados como uma coluna.

Para atingir isso, nós vamos ter que realizar esse calculo em alguns passos.

Primeiro, para cada linha nós vamos manter o campo ID_ESCOLA, fazer o derretimento dos campos de métrica e criar uma variável chamada METRICA (ex de valores: VL_APROVACAO_2005_SI_4, VL_PROJECAO_2021, ...) e os valores de VALOR.

In [ ]:
#@title Resposta
metricas = [
    "VL_APROVACAO", 
    "VL_INDICADOR_REND", 
    "VL_NOTA_MATEMATICA", 
    "VL_NOTA_PORTUGUES", 
    "VL_NOTA_MEDIA", 
    "VL_OBSERVADO", 
    "VL_PROJECAO"
]
ideb_ai = ideb_ai.melt(
    id_vars=["ID_ESCOLA"],
    value_vars=[c for c in ideb_ai if any([c.startswith(m) for m in metricas])],
    var_name="METRICA",
    value_name="VALOR"
)
ideb_af = ideb_af.melt(
    id_vars=["ID_ESCOLA"],
    value_vars=[c for c in ideb_af if any([c.startswith(m) for m in metricas])],
    var_name="METRICA",
    value_name="VALOR"
)
ideb_em = ideb_em.melt(
    id_vars=["ID_ESCOLA"],
    value_vars=[c for c in ideb_em if any([c.startswith(m) for m in metricas])],
    var_name="METRICA",
    value_name="VALOR"
)

Nós podemos aproveitar o estado atual das bases para realizar algumas limpezas de dados. 

Para reduzir o código aplicado nós vamos, em primeiro lugar, concatenar as bases em uma base única de "ideb", mas primeiro adicione um campo a cada uma delas chamada "SERIE" que deve conter os valores de "AI", "AF" e "EM" para as bases de ideb_ai, ideb_af e ideb_em respectivamente. Nós temos que fazer isso para garantir que as métricas conseguam ser associadas aos anos corretos posteriormente

In [ ]:
#@title Resposta
ideb = (
    ideb_ai.assign(SERIE="AI")
    .append(ideb_af.assign(SERIE="AF"))
    .append(ideb_em.assign(SERIE="EM"))
)

Agora nós podemos fazer alguns ajustes aos campos de valores. Segundo a documentação há 3 valores possíveis:
1. Valor numérico associado a métrica
1. "-" que indica que a escola não possuí esse dado (por exemplo se a escola não tem Anos Iniciais ela não terá coleta de IDEB para essa série)
1. "ND" que indica que o número de alunos para o qual foi feita uma coleta não é suficiente para ter um resultado com significância estatística suficiente para inferir o valor total da escola

Desta forma, para conseguir converter os valores para número vamos converter "-" e "ND" para np.nan e depois converta o campo para float64

NOTA (1): O campo ND pode, ou não, aparecer com 1 ou mais asteríscos (e.g. ND*, ND**, etc). Como você pode processar a base ao redor disso?
NOTA (2): Todos os valores estão com as casas decimais corretas para fazer a conversão?

In [ ]:
#@title Resposta
ideb = (
    ideb.assign(VALOR=lambda f: f["VALOR"].astype(str))
    .assign(VALOR=lambda f: f["VALOR"].str.replace(",", "."))
    .assign(VALOR=lambda f: f["VALOR"].str.replace("[*]", ""))
    .assign(
        VALOR=lambda f: np.where(
            f["VALOR"].str.contains("ND"),
            np.nan,
            np.where(f["VALOR"] == "-", np.nan, f["VALOR"])
        )
    )
    .assign(VALOR=lambda f: f["VALOR"].astype("float64"))
)

Ok, agora essa base tá ficando bonita!

Extraí do campo METRICA o valor de ano associado a cada uma delas, e gere uma coluna de uint16 com os valores de ANO

In [ ]:
#@title Resposta
ideb = ideb.assign(
    ANO=lambda f: f["METRICA"].str.split("_").map(
        lambda x: [c for c in x if c.isnumeric()][0]
    )
).assign(ANO=lambda f: f["ANO"].astype("uint16"))

Ótimo, agora vamos criar um campo de METRICA2, na qual nós reescreveremos o valor de METRICA excluíndo a informação de ano. Por exemplo, o valor VL_APROVACAO_2005_SI_4 deve ser reescrito como VL_APROVACAO_SI_4

In [ ]:
#@title Resposta
ideb = ideb.assign(
    METRICA2=lambda f: f.apply(
        lambda x: x["METRICA"].replace(f"_{x['ANO']}", ""),
        axis=1
    )
)

Daremos o toque final a esse campo métrica: Adicione a ele um "\_" e o valor do campo série

In [ ]:
#@title Resposta
ideb = ideb.assign(
    METRICA2=lambda f: f["METRICA2"] + "_" + f["SERIE"]
)

Finalmente, vamos reformatar a nossa base de dados. Faça uma pivot da base, onde teremos como linhas ID_ESCOLA e ANO, como colunas os dados de METRICA2 e como valores o campo VALOR.

NOTA: Lembre-se de restar os índices para a base resultante

In [ ]:
#@title Resposta
ideb = ideb.pivot_table(
    index=["ID_ESCOLA", "ANO"],
    columns="METRICA2",
    values="VALOR"
).reset_index()

Por fim, por que não converter o campo ID_ESCOLA para um inteiro?

In [ ]:
#@title Resposta
ideb["ID_ESCOLA"] = ideb["ID_ESCOLA"].astype("int")

---

### Cruzamento 

Vamos cruzar as bases de IDEB com a base de escola para que possamos fazer análises dos resultados mais a frente.

Para isso cruze as bases no ID de escola e ANO, e da base de IDEB selecione apenas as métricas de VL_OBSERVADO e VL_PROJETADO para cada série. Chame essa nova base de "cruzada"

NOTA (1): Faça um left join (qual você acha que deve ser o dataset da esquerda?)

NOTA (2): Mantenha apenas um campo de ID de escola e ANO na base

In [ ]:
#@title Resposta
cruzada = escolas.merge(
    ideb.reindex(
        columns=["ID_ESCOLA", "ANO"] 
        + [c for c in ideb if c.startswith("VL_OBS") or c.startswith("VL_PROJ")]
    ),
    left_on=["CO_ENTIDADE", "NU_ANO_CENSO"],
    right_on=["ID_ESCOLA", "ANO"],
    how="left"
).drop(columns=["ID_ESCOLA", "ANO"])

O nome dos campos da base de IDEB não é o mais intuitivo possível. Renomei os campos de VL_OBSERVADO para IDEB_{SERIE} e os campos de VL_PROJECAO para IDEB_META_{SERIE}

In [ ]:
#@title Resposta
cruzada.rename(columns={
    "VL_OBSERVADO_AF": "IDEB_AF",
    "VL_OBSERVADO_AI": "IDEB_AI",
    "VL_OBSERVADO_EM": "IDEB_EM",
    "VL_PROJECAO_AF": "IDEB_META_AF",
    "VL_PROJECAO_AI": "IDEB_META_AI",
    "VL_PROJECAO_EM": "IDEB_META_EM"
}, inplace=True)

---

## Análises 

Nós conseguimos realizar algumas análises em cima da base "cruzada".

Primeiro, imprima os valores de IDEB por série e por ano e veja quais são as primeiras conclusões que podemos tirar

In [ ]:
#@title Resposta
cruzada.groupby(["NU_ANO_CENSO"])[["IDEB_AI", "IDEB_AF", "IDEB_EM"]].mean()

# Conclusões
# IDEB de 2019 subiu com relação a 2019
# IDEB AI > IDEB AF > IDEB EM

Vamos fazer uma abertura maior. Conte o número de escolas por depêndencia administrativa e ano e calcule o valor de IDEB para cada depêndencia e ano.

In [ ]:
#@title Resposta
cruzada.groupby(["TP_DEPENDENCIA", "NU_ANO_CENSO"]).agg({
    "CO_ENTIDADE": "nunique",
    "IDEB_AI": ["mean", "count"],
    "IDEB_AF": ["mean", "count"],
    "IDEB_EM": ["mean", "count"],
})

# Escolas Municipais > Estaduais > Federais em cada um dos censos

# Olhando as séries, escolas estaduais são mais representativas em AF e EM, 
# enquanto municipais são mais em AI. Já federais são mais representativas em EM
# Isso é esperado por o ente municipal é responsável pela educação infantil
# e pelos anos iniciais. Estados e Municípios dividem a responsabilidade dos anos
# finais, e o ensino médio deve ser priorizado pelos entes estaduais

# O IDEB cresceu para todas as séries e depêndencias

# No geral IDEB Federal > IDEB Estadual > IDEB Municipal

Vamos, por fim, fazer uma análise mais profunda. Sabemos que os entes municipais são responsáveis pelos anos iniciais. Veja quais são as características de uma escola que mais se correlacionam com o IDEB AI para essa dependencia administrativa

In [ ]:
#@title Resposta
(
    cruzada.loc[lambda f: f["TP_DEPENDENCIA"] == "Municipal"]
    .drop(
        columns=[
            "CO_ENTIDADE", 
            "CO_MUNICIPIO", 
            "NU_ANO_CENSO",
            "IDEB_AF",
            "IDEB_EM",
            "IDEB_META_AI",
            "IDEB_META_AF",
            "IDEB_META_EM",
        ]
    )
    .corr()[["IDEB_AI"]]
    .assign(abs_corr=lambda f: f["IDEB_AI"].abs())
    .sort_values(by="abs_corr", ascending=False)
    .drop(columns="abs_corr")
    .head(11)
)

---

## Exportação de Dados 

Para concluir este notebook vamos exportar os dados. 

Primeiro exporte a base "escolas" como um parquet com chaves por ano na pasta deste mesmo notebook

Google Colab

In [ ]:
#@title Resposta
escolas.to_parquet(
    f"drive/MyDrive/{CAMINHO_DRIVE}/aulas/03.Manipulação de Dados/escolas.parquet",
    partition_cols=["NU_ANO_CENSO"]
)

Local

In [ ]:
#@title Resposta
escolas.to_parquet(vPATH_NOTEBOOK / "escolas.parquet", partition_cols=["NU_ANO_CENSO"])

Depois exporte a base de IDEB como um CSV com separador ";", decimal de "," e encoding de latin-1

Google Colab

In [ ]:
#@title Resposta
ideb.to_csv(
    f"drive/MyDrive/{CAMINHO_DRIVE}/aulas/03.Manipulação de Dados/ideb.csv",
    sep=";",
    decimal=",",
    encoding="latin-1"
)

Local

In [ ]:
#@title Resposta
ideb.to_csv(
    vPATH_NOTEBOOK / "ideb.csv",
    sep=";",
    decimal=",",
    encoding="latin-1",
    index=False
)

Por fim exporte a base cruzada como um arquivo pkl

Google Colab

In [ ]:
#@title Resposta
cruzada.to_pickle(f"drive/MyDrive/{CAMINHO_DRIVE}/aulas/03.Manipulação de Dados/cruzada.pkl")

Local

In [ ]:
#@title Resposta
cruzada.to_pickle(vPATH_NOTEBOOK / "cruzada.pkl")

---